#**Mixed Integer Programming**
A mixed integer linear program (MILP, MIP) is of the form

\begin{array}{ll} \mbox{minimize} & \mathbf{c^T}\mathbf{x}\\
\mbox{subject to} & \mathbf{A}\mathbf{x} \leq \mathbf{B}\\
& \mathbf{x_i} \in \mathcal Z, ∀\mathbf{i} \in \mathcal I\\
& \mathbf{x} \geq 0
\end{array}

**Example**
\begin{equation}
\begin{aligned}
\max \quad & x+y\\
\textrm{s.t.} \quad & -2x+2y \geq 1\\
 & -8x+10y \leq 13 \\
 & x,y \geq 0 \\
 & x,y \in \mathcal Z
\end{aligned}
\end{equation}
First, try to solve this problem with LP relaxation (by dropping integrality constraints).

Then find the optimal solution for Integer problem (when initializing variables put integer=True) and compare the results.

In [ ]:
import numpy as np
import cvxpy as cp

x = cp.Variable(1)
y = cp.Variable(1)

constr = [-2*x+2*y >= 1, x >= 0, y >= 0, -8*x+10*y <= 13]
problem = cp.Problem(cp.Minimize(-x-y), constr)
problem.solve()
print(x.value, y.value)

[4.] [4.5]


In [ ]:
x = cp.Variable(1, integer = True)
y = cp.Variable(1, integer = True)

constr = [-2*x+2*y >= 1, x >= 0, y >= 0, -8*x+10*y <= 13]
problem = cp.Problem(cp.Minimize(-x-y), constr)
problem.solve()
print(x.value, y.value)

[1.] [2.]


No direct way of getting from (4, 4.5) to (1, 2) by rounding!


Something more elaborate is needed: branch & bound:


1.   Assume variables are bounded, i.e., have lower and upper bounds
2.   Let $P_0$ be the initial problem, LP($P_0$) be the LP relaxation of $P_0$
3.   If in optimal solution of LP($P_0$) all integer variables take integer values then it is also an optimal solution to $P_0$
4.   Else
  *   Let $x_j$ be integer variable whose value $β_j$ at optimal solution of LP($P_0$) is such that $β_j ∉\mathcal Z$. Define
  $P_1 := P_0 ∧ x_j ≤ ⌊β_j⌋$ and $P_2 :=P_0 ∧ x_j≥⌈β_j⌉$
  *   feasibleSols($P_0$) = feasibleSols($P_1$) ∪ feasibleSols($P_2$)
  *   Idea: solve $P_1$, solve $P_2$ and then take the best
5.   We can build a binary tree of subproblems
whose leaves correspond to pending problems still to be solved
6.   This procedure terminates as integer vars have finite bounds and, at each split, the domain of $x_j$ becomes strictly smaller
7.   If LP($P_i$) has optimal solution where integer variables take integer values then solution is stored


In [ ]:
x = cp.Variable(1)
y = cp.Variable(1)

# Change the constrains using branch and bound method
constr = [-2*x+2*y >= 1, x >= 0, y >= 0, -8*x+10*y <= 13, y <= 2, x<=1]
problem = cp.Problem(cp.Minimize(-x-y), constr)
problem.solve()
print(x.value, y.value)

#**Footstep planning**
Implement footstep planning for a biped, making sure every pair of steps lands in the same H-polytope.